In [ ]:
import os
import pandas as pd
import requests
from pathlib import Path
from datetime import datetime, timedelta

# --- CONFIGURACIÓN ---
OUTPUT_DIR = Path(r"C:\Users\khora\Downloads\EstudioExodoFestividades")
BASE_URL = "https://movilidad-opendata.mitma.es/estudios_basicos/por-distritos/pernoctaciones/ficheros-diarios"

# 1. DICCIONARIO DE CIUDADES (Datos INE)
CIUDADES_INFO = {
    "Sevilla": {"prefijo": "41091", "poblacion": 684025},
    "Valencia": {"prefijo": "46250", "poblacion": 807693},
    "Pamplona": {"prefijo": "31201", "poblacion": 205762},
    "Cadiz": {"prefijo": "11012", "poblacion": 111811},
    "Tenerife": {"prefijo": "38038", "poblacion": 209395}
}

# 2. CALENDARIO DE EVENTOS (Aquí defines qué fechas mirar para cada ciudad)
# Puedes seguir ampliando esta lista con los datos de 2022, 2023, etc.
EVENTOS_A_ESTUDIAR = [
    {"ciudad": "Valencia", "evento": "Fallas", "inicio": "2024-03-15", "fin": "2024-03-19"},
    {"ciudad": "Sevilla", "evento": "Semana Santa", "inicio": "2024-03-24", "fin": "2024-03-31"},
    {"ciudad": "Sevilla", "evento": "Feria de Abril", "inicio": "2024-04-14", "fin": "2024-04-20"},
    {"ciudad": "Pamplona", "evento": "San Fermín", "inicio": "2024-07-06", "fin": "2024-07-14"},
    {"ciudad": "Cadiz", "evento": "Carnaval", "inicio": "2025-02-27", "fin": "2025-03-09"},
    {"ciudad": "Sevilla", "evento": "Semana Santa", "inicio": "2025-04-13", "fin": "2025-04-20"}
]

# --- FUNCIONES DE APOYO ---

def daterange(start_str, end_str):
    d0 = datetime.strptime(start_str, "%Y-%m-%d").date()
    d1 = datetime.strptime(end_str, "%Y-%m-%d").date()
    curr = d0
    while curr <= d1:
        yield curr
        curr += timedelta(days=1)

def download_and_convert(d):
    yyyymm = d.strftime("%Y-%m")
    yyyymmdd = d.strftime("%Y%m%d")
    url = f"{BASE_URL}/{yyyymm}/{yyyymmdd}_Pernoctaciones_distritos.csv.gz"
    parquet_path = OUTPUT_DIR / f"{yyyymmdd}.parquet"

    if parquet_path.exists(): return parquet_path

    try:
        response = requests.get(url, stream=True, timeout=60)
        response.raise_for_status()
        gz_path = OUTPUT_DIR / f"{yyyymmdd}.csv.gz"
        with open(gz_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=1024*1024): f.write(chunk)
        
        df = None
        for sep in ["|", ";", ","]:
            try:
                df = pd.read_csv(gz_path, compression="gzip", sep=sep, dtype=str)
                if df.shape[1] >= 4: break
            except: continue
        
        if df is not None:
            col_map = {"date":"fecha", "residence_area":"zona_residencia", "overnight_stay_area":"zona_pernoctacion", "people":"personas"}
            df.rename(columns=lambda x: col_map.get(x, x), inplace=True)
            df.to_parquet(parquet_path, index=False)
            gz_path.unlink()
            return parquet_path
    except Exception as e:
        print(f"  [!] Error en {yyyymmdd}: {e}")
        return None

# --- PROCESO PRINCIPAL ---

def ejecutar_estudio_segmentado():
    OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
    resultados = []

    for evento in EVENTOS_A_ESTUDIAR:
        nombre_ciudad = evento["ciudad"]
        nombre_evento = evento["evento"]
        info_ciudad = CIUDADES_INFO[nombre_ciudad]
        
        print(f"\n>>> Analizando {nombre_evento} en {nombre_ciudad} ({evento['inicio']} al {evento['fin']})")
        
        for dia in daterange(evento["inicio"], evento["fin"]):
            path_pq = download_and_convert(dia)
            if path_pq:
                df = pd.read_parquet(path_pq)
                df['personas'] = pd.to_numeric(df['personas'], errors='coerce').fillna(0)
                
                # Filtrar residentes de esa ciudad específica
                prefijo = info_ciudad["prefijo"]
                pob_total = info_ciudad["poblacion"]
                
                residentes = df[df['zona_residencia'].str.startswith(prefijo)]
                exodo = residentes[~residentes['zona_pernoctacion'].str.startswith(prefijo)]
                total_exodo = int(exodo['personas'].sum())
                
                porcentaje = round((total_exodo / pob_total) * 100, 2)
                
                resultados.append({
                    "Ciudad": nombre_ciudad,
                    "Evento": nombre_evento,
                    "Fecha": dia,
                    "Personas Fuera": total_exodo,
                    "Población Total": pob_total,
                    "% Éxodo": porcentaje
                })
                print(f"    - {dia}: {total_exodo} personas fuera ({porcentaje}%)")

    # Guardar informe
    if resultados:
        df_final = pd.DataFrame(resultados)
        df_final.to_csv(OUTPUT_DIR / "informe_festividades_segmentado.csv", index=False)
        print(f"\n--- PROCESO FINALIZADO ---")
        print(f"Informe guardado en {OUTPUT_DIR}")
        
        # Mostrar resumen de los días con más éxodo por cada evento analizado
        resumen = df_final.loc[df_final.groupby(['Ciudad', 'Evento'])['% Éxodo'].idxmax()]
        print("\nDÍAS DE MÁXIMO ÉXODO POR EVENTO:")
        print(resumen[["Ciudad", "Evento", "Fecha", "% Éxodo"]].to_string(index=False))

if __name__ == "__main__":
    ejecutar_estudio_segmentado()

In [ ]:
import pandas as pd
from pathlib import Path

# --- CONFIGURACIÓN ---
FILE_PATH = Path(r"C:\Users\khora\Downloads\EstudioExodoFestividades\20250419.parquet")
ID_SEVILLA = "41091" # Prefijo de Sevilla capital

def analizar_flujos_detallados():
    if not FILE_PATH.exists():
        print("El archivo del Sábado Santo no se encuentra. Verifica la ruta.")
        return

    df = pd.read_parquet(FILE_PATH)
    df['personas'] = pd.to_numeric(df['personas'], errors='coerce').fillna(0)

    # --- 1. ¿A DÓNDE SE VAN LOS SEVILLANOS? (ÉXODO) ---
    # Residentes en Sevilla que duermen fuera
    residentes_sevilla = df[df['zona_residencia'].str.startswith(ID_SEVILLA)]
    exodo = residentes_sevilla[~residentes_sevilla['zona_pernoctacion'].str.startswith(ID_SEVILLA)]
    
    destinos = exodo.groupby('zona_pernoctacion')['personas'].sum().reset_index()
    # Ordenar por volumen de personas
    destinos = destinos.sort_values(by='personas', ascending=False).head(10)

    # --- 2. ¿DE DÓNDE VIENEN LOS VISITANTES? (ENTRADA) ---
    # Gente que duerme en Sevilla pero NO vive allí
    pernoctan_sevilla = df[df['zona_pernoctacion'].str.startswith(ID_SEVILLA)]
    visitantes = pernoctan_sevilla[~pernoctan_sevilla['zona_residencia'].str.startswith(ID_SEVILLA)]
    
    origenes = visitantes.groupby('zona_residencia')['personas'].sum().reset_index()
    origenes = origenes.sort_values(by='personas', ascending=False).head(10)

    print("\n" + "="*40)
    print("ANÁLISIS SÁBADO SANTO SEVILLA 2025")
    print("="*40)
    
    print("\nTOP 10 DESTINOS (¿A dónde huyen los sevillanos?):")
    print(destinos.to_string(index=False))
    
    print("\nTOP 10 ORÍGENES (¿Quiénes vienen a Sevilla?):")
    print(origenes.to_string(index=False))

if __name__ == "__main__":
    analizar_flujos_detallados()